In [2]:
from model import Network
from data import load_cifar10_dataloaders, WhiteningTransformation
import torch
import torchvision.transforms as transforms
import torch.nn as nn
from criterion import ReconstructImageFromFCLoss
import numpy as np
from metrics import lpips

# reproducibility
random_seed = 2024

torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

np.random.seed(random_seed)

import optuna

In [3]:
def objective(trial):
    # load to model
    model_config = './model_config/fc1_cocktail_party_cifar10_instance.json'
    checkpoint_path = './checkpoints/122123_fc1_cocktail_party_cifar10_pretraining_wout_bias_wout_normalization.pth'
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = Network(model_config)
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['state_dict'])
    model = model.to(device)
    
    # get val loader
    batch_size = 4
    data_path = './data'
    transform = transforms.Compose([
        transforms.ToTensor(),
    ])
    
    _, val_dataloader = load_cifar10_dataloaders(data_path, batch_size, transform)
    selected_val_batch_data, selected_val_batch_label = next(iter(val_dataloader))
    selected_val_batch_data = selected_val_batch_data.to(device)
    selected_val_batch_label = selected_val_batch_label.to(device)
    
    # receiving gradients
    model.zero_grad()
    criterion = nn.CrossEntropyLoss()
    output = model(selected_val_batch_data.reshape(batch_size, -1))
    loss = criterion(output, selected_val_batch_label)
    loss.backward()
    gradient_of_layers = []
    for param in model.parameters():
        gradient_of_layers.append(param.grad.data.clone().to('cpu'))
    print([x.size() for x in gradient_of_layers])
    
    whitening_transform = WhiteningTransformation()
    whitened_gradient = torch.from_numpy(whitening_transform.transform(gradient_of_layers[0].detach().numpy().T)).to(torch.float32).T
    whitened_gradient = whitened_gradient.to(device)
    
    # criterion output testing
    unmixing_matrix = torch.rand((selected_val_batch_data.size(0), gradient_of_layers[0].size(0)), requires_grad=True, device=device)
    t_param = trial.suggest_float('t_param', 0.00001, 10)
    total_variance_loss_param = trial.suggest_float('total_variance_loss_param', 0.00001, 10)
    mutual_independence_loss_param = trial.suggest_float('mutual_independence_loss_param', 0.00001, 10)
    reconstruction_loss = ReconstructImageFromFCLoss(32, 32, 3, t_param, total_variance_loss_param, mutual_independence_loss_param)
    optimizer = torch.optim.Adam([unmixing_matrix])
    
    for iter_idx in range(25000):
        optimizer.zero_grad()
        # out_score, non_gaussianity_score, total_variance_score, mutual_independence_score
        loss, _, _, _ = reconstruction_loss(unmixing_matrix, whitened_gradient)
        loss.backward()
        optimizer.step()
        
        if (iter_idx + 1) % 1000 == 0 or iter_idx == 0:
            print('loss: {}'.format(loss.item()))
            
    unmixing_matrix = unmixing_matrix.detach().to('cpu')
    whitened_gradient = whitened_gradient.detach().to('cpu')
    
    with torch.no_grad():
        lpips_match, lpips_is_positive, mean_lpips = lpips(unmixing_matrix @ whitened_gradient, selected_val_batch_data.detach().to('cpu'), return_matches=True)
        return mean_lpips

In [4]:
num_of_trials = 100
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=num_of_trials)

[I 2023-12-25 02:42:11,628] A new study created in memory with name: no-name-02187155-59fc-4a5e-ba93-9a99fef7ba67


Files already downloaded and verified
Files already downloaded and verified
[torch.Size([10, 3072])]
loss: 44.18314743041992
loss: 9.316374778747559
loss: 9.277816772460938
loss: 9.229230880737305
loss: 9.175579071044922
loss: 9.138457298278809
loss: 9.103262901306152
loss: 9.074419021606445
loss: 9.053709983825684
loss: 9.041610717773438
loss: 9.027913093566895
loss: 9.018693923950195
loss: 9.013443946838379
loss: 9.00733470916748
loss: 8.990605354309082
loss: 8.984159469604492
loss: 8.987251281738281
loss: 8.98818302154541
loss: 8.966219902038574
loss: 8.967445373535156
loss: 8.963444709777832
loss: 8.955377578735352
loss: 8.952954292297363
loss: 8.955774307250977
loss: 8.948497772216797
loss: 8.953901290893555


[I 2023-12-25 02:43:04,378] Trial 0 finished with value: 0.1031139874830842 and parameters: {'t_param': 2.091369207776045, 'total_variance_loss_param': 7.917792917702952, 'mutual_independence_loss_param': 8.830701144870758}. Best is trial 0 with value: 0.1031139874830842.


Files already downloaded and verified
Files already downloaded and verified
[torch.Size([10, 3072])]
loss: 523.6220092773438
loss: 8.332674026489258
loss: 3.0531160831451416
loss: 2.924482583999634
loss: 2.9246795177459717
loss: 2.924445390701294
loss: 2.923374652862549
loss: 2.9241223335266113
loss: 2.921536922454834
loss: 2.9207277297973633
loss: 2.917166233062744
loss: 2.9105710983276367
loss: 2.907986879348755
loss: 2.904892921447754
loss: 2.9022977352142334
loss: 2.899380922317505
loss: 2.8969919681549072
loss: 2.8923912048339844
loss: 2.8921868801116943
loss: 2.8890938758850098
loss: 2.8852572441101074
loss: 2.8845646381378174
loss: 2.8822524547576904
loss: 2.884709596633911
loss: 2.8786284923553467
loss: 2.876858711242676


[I 2023-12-25 02:43:55,401] Trial 1 finished with value: 0.20914509892463684 and parameters: {'t_param': 6.254313653096312, 'total_variance_loss_param': 2.16721954699578, 'mutual_independence_loss_param': 2.7888110286982197}. Best is trial 0 with value: 0.1031139874830842.


...

[I 2023-12-25 03:57:10,350] Trial 97 finished with value: 0.11379032908007503 and parameters: {'t_param': 1.819035998175761, 'total_variance_loss_param': 6.88693687783544, 'mutual_independence_loss_param': 2.0495562747394973}. Best is trial 44 with value: 0.04821212589740753.


Files already downloaded and verified
Files already downloaded and verified
[torch.Size([10, 3072])]
loss: 12.024075508117676
loss: 7.025389671325684
loss: 6.989227771759033
loss: 6.958550930023193
loss: 6.936759948730469
loss: 6.92158317565918
loss: 6.909861087799072
loss: 6.899865627288818
loss: 6.8933234214782715
loss: 6.885424613952637
loss: 6.8802947998046875
loss: 6.875301837921143
loss: 6.871962070465088
loss: 6.865916728973389
loss: 6.864570140838623
loss: 6.859833240509033
loss: 6.8578619956970215
loss: 6.857293605804443
loss: 6.854398727416992
loss: 6.848634719848633
loss: 6.850628852844238
loss: 6.849428176879883
loss: 6.85026741027832
loss: 6.847599983215332
loss: 6.84454345703125
loss: 6.841332912445068


[I 2023-12-25 03:57:54,389] Trial 98 finished with value: 0.11346806213259697 and parameters: {'t_param': 0.7108749769927197, 'total_variance_loss_param': 3.4652669033785566, 'mutual_independence_loss_param': 6.810815569062923}. Best is trial 44 with value: 0.04821212589740753.


Files already downloaded and verified
Files already downloaded and verified
[torch.Size([10, 3072])]
loss: 47.64729690551758
loss: 3.7147724628448486
loss: 3.7088122367858887
loss: 3.6969716548919678
loss: 3.684251070022583
loss: 3.668072462081909
loss: 3.6524438858032227
loss: 3.6380274295806885
loss: 3.6259145736694336
loss: 3.619553565979004
loss: 3.6089961528778076
loss: 3.5950024127960205
loss: 3.5871262550354004
loss: 3.581996440887451
loss: 3.5771021842956543
loss: 3.582995653152466
loss: 3.5650365352630615
loss: 3.561464548110962
loss: 3.557607650756836
loss: 3.553396701812744
loss: 3.5526013374328613
loss: 3.5456459522247314
loss: 3.5434298515319824
loss: 3.54179048538208
loss: 3.5386698246002197
loss: 3.5340912342071533


[I 2023-12-25 03:58:40,135] Trial 99 finished with value: 0.15981220081448555 and parameters: {'t_param': 3.7331999824216564, 'total_variance_loss_param': 4.893779140632648, 'mutual_independence_loss_param': 3.455013069741858}. Best is trial 44 with value: 0.04821212589740753.
